# Shortest Path Algorithms


An implementation of a simple game consisting of a rectangular grid (of size height x width ) where each cell has a random integer value between 0 and 9. An agent starts at the upper-left corner of the grid and must reach the lower-right corner of the grid as fast as possible.


##### The algorithms made in this project use the first game mode to find the shortest path(s) on the n by k integer maze. (Time spent on a cell is the number on the cell).


#### Integer Maze and Adjacency Matrix

Below is the implementation of the functions responsible for creating the 'board' of our game, the Integer Maze and the corresponding Adjacency Matrix.

The function 'create_maze' generates a 2D array (h x w) by requesting the user to input its dimensions, (its height (h) and width (w)). 

Subsequently receiving as input the integer maze, the functions 'create_row' and 'adjacency_matrix_distance' are accountable for the manifestation of the Adjacency Matrix, taking into account that the agent can only move 1 cell up, down, left or right.

In the procedure of creating the Adjacency matrix the idea is to consider each cell of the Integer Maze as a vertex, for example a 3 by 3 matrix will have a total of 9 vertices, and the weight of the edge for moving from cell A to a neighbouring cell B will be equal to the number 'encapsulated' on cell B.

In the function create_maze, we create an Integer Maze (height x width) by letting the user choose its dimensions filled with integers from 0-9, while also checking if the user provided appropriate integer inputs, if not we throw a Value Error exception. Finally, we set the starting and end points value equal to 0 assuming agent has already spent some time (x) on starting point and 0 to the end point since agent has arrived at its destination.

Function create_row is being called by the adjacency_matrix_distance function to create the rows of the adjacency matrix.
Each row of the adjacency matrix will be equal to the number of cells of the grid initially filled with value NaN. Within the x, y loop we check the values of the Integer Maze and we replace the NaN value if the point is neighbouring and thus the agent can move to it directly. Finally, we concatenate the arrays to create a row for the 2D matrix.

Lastly, function adjacency_matrix creates the two-dimensional adjacency matrix denoting all paths between the nodes once all the necessary rows  have been created from by the 'create_row' function which takes as input the integer maze.

<b>Note: </b> vstack() is a numpy function to stack the rows vertically

In [1]:
import numpy as np
import time

def create_maze():
    height = input("\n Enter the number of rows: ")
    width = input("\n Enter the number of columns: ")
    if height.isdigit() and width.isdigit():
        height, width = int(height), int(width)
        grid = np.random.randint(0, 10, (height, width))
        grid[height - 1][width - 1] = 0
        grid[0][0] = 0
        print("\n The Integer Maze: \n")
        print(grid)
        return grid
    else:
        raise ValueError("Please provide positive integer values for the number of rows and columns")


def create_row(pos, grid):
    row_of_distance_table = np.empty((len(grid), len(grid[0])))
    row_of_distance_table[:] = np.NaN
    row = []
    for x in range(len(grid)):
        for y in range(len(grid[0])):
            if pos[0]+1 == x and pos[1] == y or pos[0]-1 == x and pos[1] == y \
                    or pos[0] == x and pos[1] == y+1 or pos[0] == x and pos[1] == y-1:
                row_of_distance_table[x][y] = grid[x][y]
    for array in row_of_distance_table:
        row = np.concatenate([row, array], axis=None)
    # print(row)
    return row


def adjacency_matrix():
    grid = create_maze()
    distance_matrix = []
    for x in range(len(grid)):
        for y in range(len(grid[0])):
            distance_matrix.append(create_row([x, y], grid))
    distance_matrix = np.vstack(distance_matrix)
    print("\n The Adjacency Matrix denoting the distances of all paths: \n")
    print(distance_matrix)
    print("\n height: ", len(distance_matrix), "width: ", len(distance_matrix[0]))
    return distance_matrix



#### Floyd-Warshall Algorithm
As my first baseline algorithm I chose to implement the Floyd-Warshall algorithm which finds the shortest path between all pairs of vertices in a weighted graph. Floyd's-Warshall's algorithm can work for directed and undirected graphs with positive and negative weighted edges but not with graphs containing negative cycles. "In graph theory, a 'cycle' in a graph is a non-empty trail in which the only repeated vertices are the first and last vertices. A 'directed cycle' in a directed graph is a non-empty directed trail in which the only repeated vertices are the first and last vertices."[^1] "A 'negative cycle' (i.e. a cycle whose edges sum to a negative value) that is reachable from the source, means that there is no cheapest path: any path that has a point on the negative cycle can be made cheaper by one more walk around the negative cycle."[^2] This implementation of the algorithm also contains a path reconstruction function, to show the shortest path found in the integer maze.

In floyd_warshall function we implement the main code for the algorithm, finding the shortest paths for all pairs of vertices.
First, we initialize the distance, and path arrays, from which we will recover the shortest path and its cost value.
In the first for loop we replace the values of each line equal to their vertex, if there is no path through a vertex k then
you can get straight from i. In the second for loop we evaluate if there is a shorter path from vertex i to vertex j through a vertex k, if there is update the shortest distance in the fw_distance_matrix, if not keep the shortest distance from i to j as it is. Furthermore, we create a path matrix of similar dimensions to pass it to the "reconstruct_path" function in order to get all the visited nodes. Each line in the adjacency matrix represents a vertex and all the paths from that vertex to the rest. If there is a distance shorter through a vertex k instead of going straight from i to j replace, the vertex in the path matrix (np arrays support accessing indexes with point notation [x, y]).

Finally we create a recursive function, constructing the path untill our destination is reached. This function takes as arguments the path_matrix, the destination which is updated untill it is equal to the source, and the path which is being constructed. We initiate our path variable as an argument because if we constructed it inside the recursive function it would be set to empty every time a recursion occurs.

<b>Note:</b> Time complexity of Floyd-Warshall algorithm is O(n^3)


[^1]:https://en.wikipedia.org/wiki/Cycle_(graph_theory)
[^2]:https://en.wikipedia.org/wiki/Bellman%E2%80%93Ford_algorithm#:~:text=If%20a%20graph%20contains%20a,walk%20around%20the%20negative%20cycle.

In [2]:
def floyd_warshall(matrix):
    vertices = len(matrix)
    fw_distance_matrix = matrix.copy()  # make a copy of matrix, (if there is no distance keep the default ones)
    fw_distance_matrix[np.isnan(fw_distance_matrix)] = np.inf  # fill indirect paths as well in fw_distance_matrix
    path_matrix = np.zeros((vertices, vertices))  # create the path matrix initially filled with 0's

    for i in range(vertices):
        for j in range(vertices):
            path_matrix[i, j] = i  # replace each line with the corresponding vertex

    for k in range(vertices):
        for i in range(vertices):
            for j in range(vertices):
                if i != j:
                    if fw_distance_matrix[i][j] > fw_distance_matrix[i][k] + fw_distance_matrix[k][j]:  # if our default value is larger replace it
                        fw_distance_matrix[i][j] = fw_distance_matrix[i][k] + fw_distance_matrix[k][j]  # update shortest distance from i to j
                        path_matrix[i, j] = path_matrix[k, j]  # update the optimal previous node 
                else:
                    fw_distance_matrix[i][j] = 0  # distances from the same node equal to 0
#     print("Floyd-Warshall distances matrix: \n")
#     print(fw_distance_matrix)
#     print("\nPath matrix: \n")
#     print(path_matrix)
    path = reconstruct_path(path_matrix, len(matrix) - 1)  # reconstruct the path to the destination node
    print("Floyd-Warshall Shortest Path: ", path, "\nCost of Shortest Path: ", fw_distance_matrix[0][len(fw_distance_matrix[0])-1])        
    return path

def reconstruct_path(path_matrix, destination, path=[]):
    source = 0
    destination = int(destination)
    if source == destination:  # return path if destination is reached
        path += [source]
        shortest_path = list(reversed(path))
        return shortest_path
    else:
        path += [destination]  # add current node       
        return reconstruct_path(path_matrix, path_matrix[source, destination])  # update destination


#### Dijkstra's Algorithm 

Dijkstra's Algorithm can be implemented in numerous different methods, in this scenario the implementation of the algorithm acquires the shortest paths from a source/starting vertex to the final destination vertex. This code can also be used to find the shortest paths from the source node to all other nodes by using the previous_node_list and printing them. Dijkstra's algorithm works for directed and undirected positive weighted graphs. The procedure of applying Dijkstra's algorithm:
- start by denoting all the vertices (in our case the grid's cells) as unvisited
- create previous_node_list with length equal to the number of vertices and initially set it's values to NaN. This list denotes all vertices with their corresponding indices and with values of previous nodes indicating the shortest path.
- create shortest_distance_from_source list representing the distances of each node from the source node.
- then loop to get the shortest distances from each node to another node while adding their total distances from the source.
- finally, the algorithm will stop when the agent will have visited all of the nodes.


In [3]:
def dijkstra(matrix, source=0):
    vertices = len(matrix)
    not_visited = np.arange(0, vertices)  # list of nodes from (0, 1, 2, 3 ..., N)
    previous_node_list = [np.nan] * vertices  # list of previous nodes for each node (how we get there)
    shortest_distance_from_source_list = [np.inf] * vertices  # table of node distances from source (values init to inf)
    shortest_distance_from_source_list[source] = 0  # value of source to source is 0

    while len(not_visited) != 0:  # visit all nodes, if yes than dijkstra's algorithm has finished
        current_node = not_visited[0]  # initialize current_node to source
        for node in not_visited:  # loop over unvisited nodes to find the shortest distance from source so far
            if shortest_distance_from_source_list[node] < shortest_distance_from_source_list[current_node]:
                current_node = node # set current node to node with shortest distance from source
        not_visited = np.delete(not_visited, np.where(not_visited == current_node))

        for i in range(vertices):
            if matrix[current_node][i] == np.NaN:  # if there is no path from current_node to node i continue
                continue
            distance = shortest_distance_from_source_list[current_node] + matrix[current_node][i]  # calculate total distance to next node
            if distance < shortest_distance_from_source_list[i]: # if distance is shorter than our current one, update it
                shortest_distance_from_source_list[i] = distance
                previous_node_list[i] = current_node # update the node which holds the shortest distance to node i
    
#     print("Dijkstra previous node list: ", previous_node_list)
#     print("\nDijkstra distance matrix: ", shortest_distance_from_source_list)
    i = vertices-1
    path = [i]
    while i != 0:
        path.append(previous_node_list[i])
        i = previous_node_list[i]
    print("Shortest Path: ", list(reversed(path)), "\nCost of Shortest Path: ", shortest_distance_from_source_list[vertices-1])  
    return shortest_distance_from_source_list, previous_node_list


#### Ant Colony Optimization (ACO) Algorithm

Ant Colony Optimization (ACO) is an algorithm influenced by the natural mechanisms found in real ant colonies, it is a metaheuristic population based probabilistic model, that uses artificial ants to provide solutions to optimization problems. In this model, artificial ants try to find the shortest path on the Integer Maze from source to destination by using the amount of pheromones and the grade of vision to adjacent destinations. While on a path, artificial ants emit a substance called 'pheromone' which evaporates over time, increasing the likelihood of consecutive ants to follow paths with increased pheromone levels. Since pheromone evaporates, shorter paths will have increased quantities of compound and will be more likely to be followed by subsequent ants compared to longer paths. Another factor that affects the decision making of ants is the distance between the points to be followed, in this case we demonstrate these two factors in two arrays called 'pheromone_matrix' and 'visibility_matrix' which are equal to the dimensions of the adjacency matrix. 

The amount of pheromone on a point = <b> (1 - rate of evaporation) * amount of pheromone on point + Dt </b>

where Dt is the amount of pheromone deposited given by <b> Dt = q/Lk </b> where Lk is the cost of the ants tour and q is a constant which determines how much will the length of the path affect the pheromone deposit. This technique is implemented in the "pheromone_update" function.

The resulting probability of an ant choosing a point between a set of neighbouring points is calculated  by:

Probability of point = <b>(pheromone on point ** a * distance of point ** b) / Sum of distance visibility and pheromone for all points </b>

This model is a "doubled-edged sword" as its randomness can provide unique solutions but also contribute to more time-consuming models.

<b>Note: </b> in1d() is a numpy function to test if each element of a one-dimensional array is present in another.

In [4]:
def pheromone_update(pheromone_matrix, path, path_distance):
    evaporation_rate = 0.001
    q = 7
    pheromone_matrix = pheromone_matrix * (1 - evaporation_rate)
    for path_move in path:
        pheromone_matrix[path_move] += q/path_distance
    return pheromone_matrix


def node_selection(matrix_row, pheromone_matrix_row, visibility_matrix_row, nodes_visited):
    nodes = np.arange(0, len(matrix_row))
    a = 1  # controls pheromones influence on the model
    b = 1  # controls visibility influence on the model
    row = (pheromone_matrix_row**a) * (visibility_matrix_row**b)  # visibility matrix = 1/distances
    probability_matrix_row = row/np.nansum(row)
    where_nans = np.isnan(probability_matrix_row)
    probability_matrix_row[where_nans] = 0  # assign a probability of 0 to non viable paths
    distance = np.nan
    neighbors = np.argwhere(~np.isnan(matrix_row))  # neighbours have non nan values
    while np.isnan(distance):
        chosen_node = np.random.choice(nodes, 1, p=probability_matrix_row)[0]  # choose move with probabilities
        if np.all(np.in1d(neighbors, nodes_visited)):  # if all neighbours have been visited and there is no other move, reset
            return ["ant_didnt_find_food"] 
        if chosen_node in nodes_visited:  # if node has already been visited, continue
            continue
        if matrix_row[chosen_node] != np.nan:  # if chosen node is valid
            distance = matrix_row[chosen_node]

    return [distance, chosen_node]


def ant_path(matrix, pheromone_matrix, visibility_matrix):
    nodes_visited = [0]
    path = []
    prev = 0
    node = 0
    total_distance = 0
    while node != len(matrix)-1:
        data = node_selection(matrix[node], pheromone_matrix[node], visibility_matrix[node], nodes_visited)
        if data[0] == "ant_didnt_find_food":  # if ant didn't find food, send it back to nest
            nodes_visited = [0]
            path = []
            prev = 0
            node = 0
            total_distance = 0
            continue
        distance = data[0]
        total_distance += distance
        node = data[1]
        path.append((prev, node))  # construct ant's path
        prev = node  # update prev node
        nodes_visited.append(node)
    return [path, total_distance]


def ant_colony(matrix):
    ant_colony_paths = []
    shortest_path = np.inf
    n_ants = 10  # choose the number of ants
    visibility_matrix = np.true_divide(1, matrix+0.1)
    pheromone_matrix = matrix.copy()
    where_non_nan = ~np.isnan(matrix)
    pheromone_matrix[where_non_nan] = 1  # initialize pheromone close to 0
    for i in range(n_ants):
        data = ant_path(matrix, pheromone_matrix, visibility_matrix)
        path = data[0]
        path_distance = data[1]
        ant_colony_paths.append((path, path_distance))
        pheromone_matrix = pheromone_update(pheromone_matrix, path, path_distance)
    for path in ant_colony_paths:
        if path[1] < shortest_path:
            shortest_path = path[1]
            show_path = path
#     print("PHEROMONE MATRIX\n", pheromone_matrix)
#     print("VISIBILITY MATRIX\n", visibility_matrix)
#     print("ANT COLONY PATHS \n", ant_colony_paths)
    print("Shortest Path: ", show_path[0], "\nCost of Shortest Path: ", show_path[1])  
    return show_path


#### Running and Testing the algorithms

<b>Note:</b> If you want a second test on the algorithms please rerun the whole notebook

In [5]:
    data_grid = adjacency_matrix()   
    
    start_time = time.time()
    print("\n ANT COLONY OPTIMIZATION ALGORITHM \n")
    ant_colony(data_grid)
    print("--- %s seconds ---" % (time.time() - start_time))
    
    start_time1 = time.time()
    print("\n DIJKSTRA'S ALGORITHM \n")
    dijkstra(data_grid)
    print("--- %s seconds ---" % (time.time() - start_time1))

    start_time2 = time.time()
    print("\n FLOYD WARSHALL ALGORITHM \n")
    floyd_warshall(data_grid)
    print("--- %s seconds ---" % (time.time() - start_time2))


 Enter the number of rows: 20

 Enter the number of columns: 20

 The Integer Maze: 

[[0 7 9 7 5 4 3 9 0 7 9 2 5 8 4 9 5 0 3 1]
 [8 0 5 0 3 8 0 1 9 2 9 1 7 8 6 6 4 2 3 0]
 [5 6 7 0 8 2 6 2 5 0 8 7 4 4 9 0 7 6 4 4]
 [8 5 7 5 4 9 8 2 1 5 5 0 0 4 8 0 3 9 8 3]
 [4 6 1 9 2 3 8 0 1 4 6 8 2 3 4 3 1 7 2 4]
 [8 2 3 8 2 2 9 5 2 3 4 3 3 2 0 5 9 3 4 1]
 [1 9 1 4 1 2 9 1 9 3 6 1 9 6 6 1 9 2 0 6]
 [6 3 3 0 2 8 3 3 5 9 0 3 8 3 1 4 4 9 4 4]
 [8 5 0 8 5 1 7 4 7 4 1 5 5 5 6 2 4 9 3 4]
 [8 5 8 1 7 7 1 3 0 0 2 1 9 3 8 1 9 4 8 6]
 [0 3 7 5 4 7 0 9 6 1 8 0 6 2 4 9 8 3 3 6]
 [8 6 1 0 7 4 1 1 1 5 9 9 2 4 4 6 2 6 5 8]
 [5 0 9 2 8 8 2 6 7 8 9 5 3 7 3 9 0 2 4 0]
 [2 3 2 4 4 9 8 3 7 2 1 7 6 3 7 7 6 1 9 8]
 [1 3 7 7 4 1 8 5 8 4 4 9 0 4 1 3 2 1 4 8]
 [2 1 7 8 8 9 7 5 4 1 0 1 6 7 3 4 3 5 6 1]
 [3 7 8 7 1 9 6 0 2 5 3 5 1 6 7 2 7 7 2 6]
 [3 0 6 5 1 0 0 7 2 4 6 7 4 2 1 3 7 1 2 9]
 [3 3 9 5 7 5 3 8 8 6 2 6 5 3 3 3 7 4 4 3]
 [2 1 8 7 2 4 6 6 6 5 7 7 2 2 5 7 5 8 2 0]]

 The Adjacency Matrix denoting the distances of all